In [51]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [52]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df = pd.read_csv(
    Path("Resources/lending_data.csv")
)

# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [53]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df["loan_status"]

# Separate the X variable, the features
X = df.drop("loan_status", axis=1)

In [54]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [55]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [56]:
# Check the balance of our target values
y.value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [57]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [58]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model = LogisticRegression(random_state=1)

# Fit the model using training data
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [59]:
# Make a prediction using the testing data
predictions = model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [60]:
# Print the balanced_accuracy score of the model
print(f'''
    Balanced Accuracy: {balanced_accuracy_score(y_test, predictions)}
''')


    Balanced Accuracy: 0.9520479254722232



In [61]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)


In [62]:
# Print the classification report for the model
cm_df = pd.DataFrame(
    cm, 
    index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
cm_df

,Predicted 0,Predicted 1
Actual 0,18663,102
Actual 1,56,563


### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** With an accuracy score of 95.2%, I feel comfortable relying on a logistic model for predicting healthy loans. However, the larger number of false positives compared to false negatives, suggests some unneccesary risk being accepted. This low precision may sway a high risk protfolio towards an accepted loan.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [63]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resample, y_resample = ros_model.fit_resample(X_train, y_train)

In [64]:
# Count the distinct values of the resampled labels data

label, count = np.unique(y_resample, return_counts=True)

for label, count in zip(label, count):
    print(f'''
        Distinct Labels in Resampled Data {label}: {count} entries.
    ''')


        Distinct Labels in Resampled Data 0: 56271 entries.
    

        Distinct Labels in Resampled Data 1: 56271 entries.
    


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [65]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
resamp_model = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
resamp_model.fit(X_resample, y_resample)

# Make a prediction using the testing data
predictions = resamp_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [66]:
# Print the balanced_accuracy score of the model 
print(f'''
    Balanced Accuracy Score: {balanced_accuracy_score(y_test, predictions)}
''')


    Balanced Accuracy Score: 0.9936781215845847



In [67]:
# Generate a confusion matrix for the model
resmp_cm = confusion_matrix(y_test, predictions)

In [68]:
# Print the classification report for the model
resmp_cm_df = pd.DataFrame(
    resmp_cm,
    index = ["Actual 0", "Actual 1"],
    columns = ["Predicted 0", "Predicted 1"]
)
resmp_cm_df

,Predicted 0,Predicted 1
Actual 0,18649,116
Actual 1,4,615


### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** With greater representation in minority classes, recall increases greatly with fewer false negatives accounted for in the resampled model. The model mistakenly labels low risk loans as high risk at a lower rate after employing the RandomOverSample module. It is more difficult to see if precision is improved in a significant way without directly measuring significance. However, a rise in True Positives for low risk loans suggests an increase in precision. 